In [1]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, SimpleRNN

Using TensorFlow backend.


In [0]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

# Embedding dimensions.
row_hidden = 128
col_hidden = 128

In [3]:
# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


In [4]:
# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
vector, dim, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(vector, dim, pixel))


W0716 11:34:20.701930 140190970021760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 11:34:20.737976 140190970021760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [7]:
# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(SimpleRNN(row_hidden))(x)


W0716 11:34:24.048410 140190970021760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
# Encodes columns of encoded rows.
encoded_columns = SimpleRNN(col_hidden)(encoded_rows)

In [9]:
# Final predictions and model.
prediction = Dense(10, activation='softmax')(encoded_columns)
model = Model(inputs=x, outputs=prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

W0716 11:34:28.059951 140190970021760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0716 11:34:28.082912 140190970021760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 28, 128)           16640     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 50,826
Trainable params: 50,826
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

W0716 11:34:37.328736 140190970021760 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0716 11:34:37.610990 140190970021760 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 179s 3ms/step - loss: 0.4812 - acc: 0.8541 - val_loss: 0.2328 - val_acc: 0.9342
Epoch 2/5
60000/60000 [==============================] - 178s 3ms/step - loss: 0.2516 - acc: 0.9274 - val_loss: 0.2978 - val_acc: 0.9137
Epoch 3/5
60000/60000 [==============================] - 178s 3ms/step - loss: 0.2212 - acc: 0.9369 - val_loss: 0.1570 - val_acc: 0.9556
Epoch 4/5
60000/60000 [==============================] - 179s 3ms/step - loss: 0.2038 - acc: 0.9427 - val_loss: 0.1882 - val_acc: 0.9462
Epoch 5/5
60000/60000 [==============================] - 179s 3ms/step - loss: 0.2050 - acc: 0.9440 - val_loss: 0.1696 - val_acc: 0.9525


In [12]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.16955553996562958
Test accuracy: 0.9525
